# Road Safety in San Diego County

In [1]:
!pip install geojson

In [2]:
#imports 

# Display plots directly in the notebook instead of in a new window
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string

In [3]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

## Data frame management

We should figure out how we want to structure the data frames. 
Options include: 
- Merge into one df
- Find matching identifiers and try to merge data by these
- Work in completely independent dfs

In [4]:
df_police_stops = pd.read_csv("/Users/sarahamiraslani/Desktop/Police Stops.csv")
df_collisions = pd.read_csv("/Users/sarahamiraslani/Desktop/SD collisions.csv")
df_police_calls = pd.read_csv("/Users/sarahamiraslani/Desktop/Police Calls 2019.csv")


/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Cleaning

In [5]:
# Change entries in col 'day_of_week' to represent the day of the week in words
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(1,'Monday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(2,'Tuesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(3,'Wednesday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(4,'Thursday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(5,'Friday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(6,'Saturday')
df_police_calls['day_of_week'] = df_police_calls['day_of_week'].replace(7,'Sunday')

In [6]:
# Splitting datetime for the datasets
date_time = df_collisions['date_time'].str.split(' ', n=1, expand=True)
df_collisions = df_collisions.drop(['date_time'], axis=1)
df_collisions['date'] = date_time[0]
df_collisions['time'] = date_time[1]


date_time = df_police_calls['date_time'].str.split(' ', n=1, expand=True)
df_police_calls = df_police_calls.drop(['date_time'], axis=1)
df_police_calls['date'] = date_time[0]
df_police_calls['time'] = date_time[1]

In [7]:
# dropping useless columns

drop_cols_collision = ['address_pd_primary','address_pd_intersecting', 'address_name_intersecting', 'address_sfx_intersecting','report_id']
drop_cols_calls = ['address_dir_primary', 'address_dir_intersecting','incident_num']
drop_cols_stops = ['isstudent', 'gend_nc','agency','stop_id']

df_collisions = df_collisions.drop(drop_cols_collision, axis = 1)
df_police_calls = df_police_calls.drop(drop_cols_calls,axis = 1)
df_police_stops = df_police_stops.drop(drop_cols_stops, axis = 1)

In [8]:
# Making everything lowercase

df_collisions['address_road_primary'] = df_collisions['address_road_primary'].str.lower()
df_collisions['address_sfx_primary'] = df_collisions['address_sfx_primary'].str.lower()
df_collisions['charge_desc'] = df_collisions['charge_desc'].str.lower()
df_collisions['hit_run_lvl'] = df_collisions['hit_run_lvl'].str.lower()

df_police_calls['address_road_primary']=df_police_calls['address_road_primary'].str.lower()
df_police_calls['address_road_intersecting']=df_police_calls['address_road_intersecting'].str.lower()
df_police_calls['address_sfx_primary'] = df_police_calls['address_sfx_primary'].str.lower()

df_police_stops['address_city'] = df_police_stops['address_city'].str.lower()


We can probably merge **address_road_primary** and **address_sfx_primary**

In [9]:
df_collisions.head()

,police_beat,address_number_primary,address_road_primary,address_sfx_primary,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl,date,time
0,935,5500,valerio,trail,MISC-HAZ,VC,miscellaneous hazardous violations of the vehi...,0,0,misdemeanor,2017-01-01,00:01:00
1,322,6400,crawford,street,MISC-HAZ,VC,miscellaneous hazardous violations of the vehi...,0,0,misdemeanor,2017-01-01,00:01:00
2,124,8300,cam del oro,,MISC-HAZ,VC,miscellaneous hazardous violations of the vehi...,0,0,misdemeanor,2017-01-01,00:01:00
3,325,8100,royal gorge,drive,22107,VC,turning movements and required signals,0,0,misdemeanor,2017-01-01,00:01:00
4,521,1000,11th,avenue,22107,VC,turning movements and required signals,0,0,misdemeanor,2017-01-01,01:00:00


In [10]:
df_police_calls.head()

,day_of_week,address_number_primary,address_road_primary,address_sfx_primary,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority,date,time
0,Monday,3800,national,ave,NaN,NaN,AU1,DUP,441,1,2019-01-01,00:00:01
1,Monday,6500,reflection,dr,NaN,NaN,415,K,321,2,2019-01-01,00:00:04
2,Monday,0,06th,ave,island,NaN,FD,K,523,2,2019-01-01,00:00:09
3,Monday,0,vesta,st,main,NaN,AU1,W,511,1,2019-01-01,00:00:19
4,Monday,0,04th,ave,island,NaN,586,CAN,523,4,2019-01-01,00:00:31


In [11]:
df_police_stops.head()

,ori,exp_years,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,intersection,address_block,...,address_city,beat,beat_name,pid,perceived_limited_english,perceived_age,perceived_gender,gender_nonconforming,gend,perceived_lgbt
0,CA0371100,10,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",NaN,700.0,...,san diego,122,Pacific Beach 122,1,0,25,Male,0,1,No
1,CA0371100,18,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",I-5,NaN,...,san diego,121,Mission Beach 121,1,0,25,Male,0,1,No
2,CA0371100,1,2018-07-01,00:05:43,15,1,10,Other,NaN,4400.0,...,san diego,822,El Cerrito 822,1,0,30,Male,0,1,No
3,CA0371100,1,2018-07-01,00:05:43,15,1,10,Other,NaN,4400.0,...,san diego,822,El Cerrito 822,2,0,30,Female,0,2,No
4,CA0371100,3,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",NaN,4800.0,...,san diego,614,Ocean Beach 614,1,0,23,Male,0,1,No


In [13]:
df_police_stops['perceived_limited_english'].describe()
df_police_stops['gender_nonconforming'].describe()

count    1.80e+05
mean     6.62e-04
std      2.57e-02
min      0.00e+00
25%      0.00e+00
50%      0.00e+00
75%      0.00e+00
max      1.00e+00
Name: gender_nonconforming, dtype: float64

visualization tools: https://medium.com/@stallonejacob/d3-in-juypter-notebook-685d6dca75c8

## Geospatial tools
- Geojson: https://pypi.org/project/geojson/
    - https://www.datacamp.com/community/tutorials/geospatial-data-python
- GeoPandas: http://geopandas.org/